In [1]:
from allennlp.predictors.predictor import Predictor
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)

error loading _jsonnet (this is expected on Windows), treating C:\Users\srina\AppData\Local\Temp\tmphqyxowkv\config.json as plain json
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import itertools
import re
import nltk
import string
import pandas as pd
import numpy as np
import rouge
ROUGE = rouge.Rouge(metrics=['rouge-n'],
                           max_n=1)
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from nltk.corpus import stopwords
from collections import OrderedDict

In [13]:
df = pd.read_csv('datasets/SummarizationCSTitleAbstract03.csv', lineterminator='\n')

In [14]:
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.head(100)
df.head()

,title,abstract
0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ..."
1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...
2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...
3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...
4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...


In [15]:
# import json
# from pycorenlp import StanfordCoreNLP

# nlp = StanfordCoreNLP('http://localhost:9000')


# def resolve(corenlp_output):
#     """ Transfer the word form of the antecedent to its associated pronominal anaphor(s) """
#     for coref in corenlp_output['corefs']:
#         mentions = corenlp_output['corefs'][coref]
#         antecedent = mentions[0]  # the antecedent is the first mention in the coreference chain
#         for j in range(1, len(mentions)):
#             mention = mentions[j]
#             if mention['type'] == 'PRONOMINAL':
#                 # get the attributes of the target mention in the corresponding sentence
#                 target_sentence = mention['sentNum']
#                 target_token = mention['startIndex'] - 1
#                 # transfer the antecedent's word form to the appropriate token in the sentence
#                 corenlp_output['sentences'][target_sentence - 1]['tokens'][target_token]['word'] = antecedent['text']


# def get_resolved(corenlp_output):
#     """Return the "resolved" output sentence"""
#     possessives = ['hers', 'his', 'their', 'theirs']
#     output_sentence = ""  # Empty string to accumulate the output sentence
    
#     for sentence in corenlp_output['sentences']:
#         for token in sentence['tokens']:
#             output_word = token['word']
#             if token['lemma'] in possessives or token['pos'] == 'PRP$':
#                 output_word += "'s"
#             output_word += token['after']
#             output_sentence += output_word  # Append the output_word to the output_sentence
    
#     return output_sentence

In [16]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def sentence_segment(self, doc, lower):
        sentences = []
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sents = re.split(pattern, doc)
        sents = [sentence.split() for sentence in sents if any(char.isalpha() for char in sentence)]
        
        for sent in sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if lower is True:
                    selected_words.append(token.lower())
                else:
                    selected_words.append(token)
            sentences.append(selected_words)
        return sentences
        
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                window_size=4, lower=False):
        """Main function to analyze text"""
        
        # Filter sentences
        sentences = self.sentence_segment(text, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [175]:
def summarize_text(dataframe, weights):
    # Filling in Nan values
    dataframe['abstract'] = dataframe['abstract'].fillna('This abstract does not exist')

    # Getting tf-idf tables
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(dataframe['abstract'])
    feature_names = vectorizer.get_feature_names_out()

    # Generating best title for each abstract
    summaries = []
    scores = []
    resolved_abstracts = []
    for i in range(len(dataframe)):
        abstract = dataframe.loc[i, 'abstract']
        #print('\nOriginal:', abstract)
        
#         # Stanford Anaphora Resolution
#         output = nlp.annotate(abstract, properties= {'annotators':'dcoref','outputFormat':'json','ner.useSUTime':'false'})
#         output = json.loads(output)
#         resolve(output)
#         abstract = get_resolved(output)
#         print('\nResolved:', abstract)
        
        # Allen Anaphora Resolution
        prediction = predictor.predict(document=abstract)  # get prediction
        abstract = predictor.coref_resolved(abstract)  # resolved text
        resolved_abstracts.append(abstract)
#         print('\nResolved:', abstract)
        
        title = dataframe.loc[i, 'title']
        tfidf_scores = tfidf_matrix[i].toarray().flatten()
        
        textRank = TextRank4Keyword()
        textRank.analyze(abstract)
        text_rank_scores = textRank.node_weight

        # Sentence Tokenization
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sentences = re.split(pattern, abstract)
        sentences = [sentence for sentence in sentences if any(char.isalpha() for char in sentence)]

        # N-gram scoring based on tf-idf values
        max_score = -1
        for sentence in sentences:
            words = sentence.split(' ')
            ngrams_list = list(ngrams(words, len(title.split())))
            score = 0
            for ngram in ngrams_list:
                tfidf_score = sum(tfidf_scores[vectorizer.vocabulary_.get(word.lower(), -1)] for word in ngram)
                tr_score = sum(text_rank_scores[word] if word in text_rank_scores else 0.0 for word in ngram)

#                 if ngram in list(text_rank_scores.keys()):
#                     tr_score = (text_rank_scores[ngram])
#                 else:
#                     tr_score = 0.0
                    
                # Combine scores using weights
                score = weights['tfidf'] * tfidf_score + weights['textrank'] * tr_score
                if (score > max_score):
                    max_score = score
                    summary = ' '.join(ngram)
                    
                wordlist = words
                stop_words = set(stopwords.words('english'))
                wordlist = [w for w in wordlist if not w in stop_words]

                tagged = nltk.pos_tag(wordlist)

                for i in tagged:
                    if i[1][0] == 'N' and i in wordlist:
                        s[i[0]] += weights['nouns']

        # Best n-gram is taken as title
        summaries.append(summary)
        # Rouge-1 score calculation
        score = ROUGE.get_scores(summary, title)
        p = score["rouge-1"]["p"]
        r = score["rouge-1"]["r"]
        f1 = score["rouge-1"]["f"]
        scores.append([p, r, f1])

    dataframe['resolved abstract'] = resolved_abstracts
    dataframe['summary'] = summaries
    dataframe['rouge-1 score'] = scores
    
    return dataframe, s

In [18]:
def grid_search_weights(dataframe, weight_ranges):
    best_weights = None
    best_score = -1

    # Generate all possible combinations of weights within the specified ranges
    weight_combinations = list(itertools.product(*weight_ranges.values()))

    for weights in weight_combinations:
        # Set the weights for each feature
        weight_dict = {feature: weight for feature, weight in zip(weight_ranges.keys(), weights)}

        # Summarize text using the current weights
        summaries, s = summarize_text(dataframe, weights=weight_dict)

        # Evaluate the performance using an appropriate evaluation metric
        score = evaluate_summaries(summaries,'rouge-1 score')

        # Keep track of the best weights and score
        if score > best_score:
            best_score = score
            best_weights = weight_dict

    return best_weights, best_score

In [19]:
def evaluate_summaries(dataframe, col):
    p_sum = 0
    for score in dataframe[col]:
        p_sum += score[0]
    total = len(dataframe[col])
    avg = p_sum/total
    return avg

In [20]:
weight_ranges = {
    'tfidf': [0.2, 0.4, 0.6, 0.8, 1.0],
    'nouns': [0.1, 0.2, 0.3, 0.4, 0.5, 1.0],
    'textrank': [0.1, 0.2, 0.3, 0.4, 0.5, 1.0]
}

In [21]:
weights = grid_search_weights(df.head(10), weight_ranges)[0]

C:\Users\srina\AppData\Local\Temp\ipykernel_23532\3243153208.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['abstract'] = dataframe['abstract'].fillna('This abstract does not exist')
C:\Users\srina\AppData\Local\Temp\ipykernel_23532\3243153208.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['resolved abstract'] = resolved_abstracts
C:\Users\srina\AppData\Local\Temp\ipykernel_23532\3243153208.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [22]:
weights

{'tfidf': 0.2, 'nouns': 0.1, 'textrank': 0.1}

In [185]:
df = pd.read_csv('datasets/SummarizationCSTitleAbstract03.csv', lineterminator='\n')
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.head(100)
df.head()

,title,abstract
0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ..."
1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...
2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...
3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...
4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...


In [186]:
df, s = summarize_text(df, weights)

C:\Users\srina\OneDrive\Desktop\LinuxShared\NLP\Text Summarization\projectsum\sumenv\lib\site-packages\allennlp\modules\token_embedders\pretrained_transformer_embedder.py:385: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  num_effective_segments = (seq_lengths + self._max_length - 1) // self._max_length


In [187]:
# df = summarize_text(df)

In [188]:
df

,title,abstract,resolved abstract,summary,rouge-1 score
0,A Microbiological Survey and Characterization,"In our study, two dairy compost heaps and one ...","In our study, two dairy compost heaps and one ...",and surface locations of the,"[0.2, 0.2, 0.20000000000000004]"
1,eb 2 00 6 Non-Commutative Formal Groups in Pos...,We describe geometric non-commutative formal g...,We describe geometric non-commutative formal g...,the symmetric Sg and the associated graded alg...,"[0.0, 0.0, 0.0]"
2,An Alternating Mesh Quality Metric Scheme for ...,In the numerical solution of partial different...,In the numerical solution of partial different...,the mesh quality within the optimization proce...,"[0.18181818181818182, 0.18181818181818182, 0.1..."
3,Researching Distance Learning Experiences Usin...,Qualitative case study is hardly a research te...,Qualitative case study is hardly a research te...,an overview of the study before describing the...,"[0.2, 0.21428571428571427, 0.20689655172413796]"
4,Un Motor de Transformación de Modelos con Sopo...,Resumen. En la actualidad están apareciendo un...,Resumen. En la actualidad están apareciendo un...,la arquitectura de un motor de transformación ...,"[0.6428571428571429, 0.6428571428571429, 0.642..."
...,...,...,...,...,...
95,ExoMol molecular line lists-XXVI : spectra of ...,Line lists for the sulphur-containing molecule...,Line lists for the sulphur-containing molecule...,radical) and NS are shown and comparisons made...,"[0.2, 0.2, 0.20000000000000004]"
96,Multiround Private Information Retrieval: Capa...,Private information retrieval (PIR) is the pro...,Private information retrieval (PIR) is the pro...,capacity of Private information retrieval (PIR...,"[0.5, 0.5, 0.5]"
97,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,Efficient Integral Equation Algorithms and The...,of low frequency magnetic fields are discussed...,"[0.1, 0.1, 0.10000000000000002]"
98,Are the Determinants of Markup Size Industry-S...,The aim of this paper is to identify factors t...,The aim of this paper is to identify factors t...,affect the pricing policy in Slovenian manufac...,"[0.5384615384615384, 0.5, 0.5185185185185186]"


In [189]:
print(df['abstract'][2])

In the numerical solution of partial differential equations (PDEs), high-quality meshes are crucial for the stability, accuracy, and convergence of the associated PDE solver. Mesh quality improvement is often performed to improve the quality of meshes before use in numerical solution of the PDE. Mesh smoothing (performed via optimization) is one popular technique for improving the mesh quality; it does so by making adjustments to the vertex locations. When an inefficient mesh quality metric is used to design the optimization problem, and hence also to measure the mesh quality within the optimization procedure, convergence of the optimization method can be much slower than desired. However, for many applications, the choice of mesh quality metric and the optimization problem should be considered fixed. In this paper, we propose a simple mesh quality metric alternation scheme for use in the mesh optimization process. The idea is to alternate the use of the original inefficient mesh quali

In [190]:
print(df['summary'][2])

the mesh quality within the optimization procedure, convergence of the optimization


In [191]:
print(df['title'][2])

An Alternating Mesh Quality Metric Scheme for Efficient Mesh Quality Improvement


In [192]:
p_sum = r_sum = f1_sum = 0
for score in df['rouge-1 score']:
    p_sum += score[0]
    r_sum += score[1]
    f1_sum += score[2]
total = len(df['rouge-1 score'])
avg = [p_sum/total, r_sum/total, f1_sum/total]
print(avg)

[0.26319691857849753, 0.26249436272511384, 0.2621993591475638]


In [193]:
# Convert DataFrame to CSV file
df.to_csv('combined.csv', index=False)